In [6]:
# packages
import pandas as pd
from urllib.parse import urlparse,urlencode
import re
from bs4 import BeautifulSoup
import requests
import whois
import urllib.request
from datetime import datetime
import time
import socket
from urllib.error import HTTPError
from cython.parallel import prange

In [7]:
from datetime import datetime
itime = datetime.now()
for i in prange(0,10000):
    pass
ftime = datetime.now()
print(ftime-itime)

0:00:00.000998


In [8]:
computer_raw_data=pd.read_csv("new_legitimate_urls.txt",header=None,names=['urls'])
computer_raw_data.head(10)

,urls
0,http://www.liquidgeneration.com/
1,http://www.ceres.dti.ne.jp/~nekoi/senno/senfir...
2,http://www.galeon.com/kmh/
3,http://www.fanworkrecs.com/
4,http://www2.117.ne.jp/~mb1996ax/enadc.html
5,http://archive.rhps.org/fritters/yui/index.html
6,http://www.cutepet.org/
7,http://www.taremeparadise.com/
8,http://www.internetdump.com/users/pornographit...
9,http://darkkaminari.net


In [9]:
rows = len(computer_raw_data)
rows

982

In [10]:
cd = None
class FeatureExtraction:
    def __init__(self):
        pass
    
    def getProtocol(self,url):
        return urlparse(url).scheme
    
    def getDomain(self,url):
        return urlparse(url).netloc
    
    def getPath(self,url):
        return urlparse(url).path
    
    def havingIP(self,url):
        """If the domain part has IP then it is phishing otherwise legitimate"""
        match=re.search('(([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\/)|'  #IPv4
                    '((0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\/)'  #IPv4 in hexadecimal
                    '(?:[a-fA-F0-9]{1,4}:){7}[a-fA-F0-9]{1,4}',url)     #Ipv6
        if match:
            #print match.group()
            return 1            # phishing
        else:
            #print 'No matching pattern found'
            return 0            # legitimate
    
    def long_url(self,url):
        """This function is defined in order to differntiate website based on the length of the URL"""
        if len(url) < 54:
            return 0            # legitimate
        elif len(url) >= 54 and len(url) <= 75:
            return 2            # suspicious
        else:
            return 1            # phishing
    
    def have_at_symbol(self,url):
        """This function is used to check whether the URL contains @ symbol or not"""
        if "@" in url:
            return 1            # phishing
        else:
            return 0            # legitimate
    
    def redirection(self,url):
        """If the url has symbol(//) after protocol then such URL is to be classified as phishing """
        if "//" in urlparse(url).path:
            return 1            # phishing
        else:
            return 0            # legitimate
        
    def prefix_suffix_separation(self,url):
        """If the domain has '-' symbol then it is considered as phishing site"""
        if "-" in urlparse(url).netloc:
            return 1            # phishing
        else:
            return 0            # legitimate
        
    def sub_domains(self,url):
        """If the url has more than 3 dots then it is a phishing"""
        if url.count(".") < 3:
            return 0            # legitimate
        elif url.count(".") == 3:
            return 2            # suspicious
        else:
            return 1            # phishing
        
    def shortening_service(self,url):
        """Tiny URL -> phishing otherwise legitimate"""
        match=re.search('bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|'
                    'yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|'
                    'short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|'
                    'doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|'
                    'db\.tt|qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|'
                    'q\.gs|is\.gd|po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|'
                    'x\.co|prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|tr\.im|link\.zip\.net',url)
        if match:
            return 1               # phishing
        else:
            return 0               # legitimate
        
    def web_traffic(self,url):
        try:
            rank = BeautifulSoup(urllib.request.urlopen("http://data.alexa.com/data?cli=10&dat=s&url=" + url).read(), "xml").find("REACH")['RANK']
        except TypeError:
            return 1
        except HTTPError:
            return 2
        rank= int(rank)
        if (rank<100000):
            return 0
        else:
            return 2
        
    def domain_registration_length(self,url):
        dns = 0
        try:
            domain_name = whois.whois(urlparse(url).netloc)
        except:
            dns = 1
        
        if dns == 1:
            return 1      #phishing
        else:
            expiration_date = domain_name.expiration_date
            today = time.strftime('%Y-%m-%d')
            today = datetime.strptime(today, '%Y-%m-%d')
            if expiration_date is None:
                return 1
            elif type(expiration_date) is list or type(today) is list :
                return 2     #If it is a type of list then we can't select a single value from list. So,it is regarded as suspected website  
            else:
                creation_date = domain_name.creation_date
                expiration_date = domain_name.expiration_date
                if (isinstance(creation_date,str) or isinstance(expiration_date,str)):
                    try:
                        creation_date = datetime.strptime(creation_date,'%Y-%m-%d')
                        expiration_date = datetime.strptime(expiration_date,"%Y-%m-%d")
                    except:
                        return 2
                registration_length = abs((expiration_date - today).days)
                if registration_length / 365 <= 1:
                    return 1 #phishing
                else:
                    return 0 # legitimate
            
    def age_domain(self,url):
        dns = 0
        try:
            domain_name = whois.whois(urlparse(url).netloc)
        except:
            dns = 1
        
        if dns == 1:
            return 1
        else:
            creation_date = domain_name.creation_date
            expiration_date = domain_name.expiration_date
            if (isinstance(creation_date,str) or isinstance(expiration_date,str)):
                try:
                    creation_date = datetime.strptime(creation_date,'%Y-%m-%d')
                    expiration_date = datetime.strptime(expiration_date,"%Y-%m-%d")
                except:
                    return 2
            if ((expiration_date is None) or (creation_date is None)):
                return 1
            elif ((type(expiration_date) is list) or (type(creation_date) is list)):
                return 2
            else:
                ageofdomain = abs((expiration_date - creation_date).days)
                if ((ageofdomain/30) < 6):
                    return 1
                else:
                    return 0
     
    
    def dns_record(self,url):
        dns = 0
        try:
            domain_name = whois.whois(urlparse(url).netloc)
            #rint(domain_name)
        except:
            dns = 1
        
        if dns == 1:
            return 1
        else:
            return 0
        
   
    def statistical_report(self,url):
        hostname = url
        h = [(x.start(0), x.end(0)) for x in re.finditer('https://|http://|www.|https://www.|http://www.', hostname)]
        z = int(len(h))
        if z != 0:
            y = h[0][1]
            hostname = hostname[y:]
            h = [(x.start(0), x.end(0)) for x in re.finditer('/', hostname)]
            z = int(len(h))
            if z != 0:
                hostname = hostname[:h[0][0]]
        url_match=re.search('at\.ua|usa\.cc|baltazarpresentes\.com\.br|pe\.hu|esy\.es|hol\.es|sweddy\.com|myjino\.ru|96\.lt|ow\.ly',url)
        try:
            ip_address = socket.gethostbyname(hostname)
            ip_match=re.search('146\.112\.61\.108|213\.174\.157\.151|121\.50\.168\.88|192\.185\.217\.116|78\.46\.211\.158|181\.174\.165\.13|46\.242\.145\.103|121\.50\.168\.40|83\.125\.22\.219|46\.242\.145\.98|107\.151\.148\.44|107\.151\.148\.107|64\.70\.19\.203|199\.184\.144\.27|107\.151\.148\.108|107\.151\.148\.109|119\.28\.52\.61|54\.83\.43\.69|52\.69\.166\.231|216\.58\.192\.225|118\.184\.25\.86|67\.208\.74\.71|23\.253\.126\.58|104\.239\.157\.210|175\.126\.123\.219|141\.8\.224\.221|10\.10\.10\.10|43\.229\.108\.32|103\.232\.215\.140|69\.172\.201\.153|216\.218\.185\.162|54\.225\.104\.146|103\.243\.24\.98|199\.59\.243\.120|31\.170\.160\.61|213\.19\.128\.77|62\.113\.226\.131|208\.100\.26\.234|195\.16\.127\.102|195\.16\.127\.157|34\.196\.13\.28|103\.224\.212\.222|172\.217\.4\.225|54\.72\.9\.51|192\.64\.147\.141|198\.200\.56\.183|23\.253\.164\.103|52\.48\.191\.26|52\.214\.197\.72|87\.98\.255\.18|209\.99\.17\.27|216\.38\.62\.18|104\.130\.124\.96|47\.89\.58\.141|78\.46\.211\.158|54\.86\.225\.156|54\.82\.156\.19|37\.157\.192\.102|204\.11\.56\.48|110\.34\.231\.42',ip_address)  
        except:
            return 1

        if url_match:
            return 1
        else:
            return 0
        
    def https_token(self,url):
        match=re.search('https://|http://',url)
        try:
            if match.start(0)==0 and match.start(0) is not None:
                url=url[match.end(0):]
                match=re.search('http|https',url)
                if match:
                    return 1
                else:
                    return 0
        except:
            return 1




In [11]:
#features
protocol = []
domain = []
path = []
having_ip = []
len_url = []
having_at_symbol = []
redirection_symbol = []
prefix_suffix_separation = []
sub_domains = []
tiny_url = []
abnormal_url = []
web_traffic = []
domain_registration_length = []
dns_record = []
statistical_report = []
age_domain = []
http_tokens = []

In [12]:
# object creation
fe = FeatureExtraction()
rows = len(computer_raw_data["urls"])

for i in range(0,rows):
    url=computer_raw_data["urls"][i]
    print(i ),print(url)
    protocol.append(fe.getProtocol(url))
    path.append(fe.getPath(url))
    domain.append(fe.getDomain(url))
    having_ip.append(fe.havingIP(url))
    len_url.append(fe.long_url(url))
    having_at_symbol.append(fe.have_at_symbol(url))
    redirection_symbol.append(fe.redirection(url))
    prefix_suffix_separation.append(fe.prefix_suffix_separation(url))
    sub_domains.append(fe.sub_domains(url))
    tiny_url.append(fe.shortening_service(url))
    web_traffic.append(fe.web_traffic(url))
    domain_registration_length.append(fe.domain_registration_length(url))
    dns_record.append(fe.dns_record(url))
    statistical_report.append(fe.statistical_report(url))
    age_domain.append(fe.age_domain(url))
    http_tokens.append(fe.https_token(url))

0
http://www.liquidgeneration.com/
1
http://www.ceres.dti.ne.jp/~nekoi/senno/senfirst.html
2
http://www.galeon.com/kmh/
3
http://www.fanworkrecs.com/
4
http://www2.117.ne.jp/~mb1996ax/enadc.html
5
http://archive.rhps.org/fritters/yui/index.html
6
http://www.cutepet.org/
7
http://www.taremeparadise.com/
8
http://www.internetdump.com/users/pornographite/index1.html
9
http://darkkaminari.net
10
http://www.iei.net/~bkos1/velneko.htm
11
http://www.jasonmeador.com/
12
http://www.geocities.com/kaseychan17/index.html
13
http://www.angelfire.com/journal/coldlemonade/index.html
14
http://e.webring.com/hub?sid=&amp;ring=hentff98&amp;id=&amp;list
15
http://www.nemurokinenkan.net
16
http://j-heaven.tripod.com/library.htm
17
http://www.angelfire.com/poetry/nicolesstories/
18
http://thesheeparecoming.tripod.com/papercrane/
19
http://www.geocities.com/eddie_monsoon2002/
20
http://www.shokushu.com/
21
http://www.geocities.com/timessquare/arcade/7810/p_hgames.htm
22
http://www.hentaisocialclub.com/gangu

175
http://citypaper.net/articles/022996/article001.shtml
176
http://jam.canoe.ca/movies/artists/g/gottfried_gilbert/1997/02/21/758674.html
177
http://www.voicechasers.com/database/showactor.php?actorid=2148
178
http://www.kylehebert.com/
179
http://www.post-gazette.com/magazine/19990223voicetalent1.asp
180
http://www.serkworks.com/roommates/index.htm
181
http://www.armory.com/~keeper/jesshirt.html
182
http://www.voicechasers.com/database/showactor.php?actorid=1220
183
http://docharris.com/
184
http://www.lushmedia.co.uk/
185
http://www.dav-productions.com/
186
http://www.ccvideo.net/
187
http://www.blueyed.com/
188
http://www.koalatv.com/
189
http://www.goaggressive.com/
190
http://www.emblemma.com/
191
http://www.killingtimeproductions.com/
192
http://www.rollingrproductions.com/
193
http://www.boutwellstudios.com
194
http://www.blindruckus.com
195
http://www.roadrecords.com/clientlist.html
196
http://www.coupestudios.com
197
http://www.kellogg-rice.com
198
http://www.sarley.com
199


407
http://www.kaska.demon.co.uk/
408
http://www.nestersoft.com/timeleft/
409
http://www.chemware.co.nz/tic.htm
410
http://www.blueprintsoftwareworks.com/atomsync/
411
http://www.hoursandminutes.com/
412
http://www.hiclock.com/
413
http://www.springcreeksoftware.com/
414
http://timerecorder.51.net/
415
http://www.rayslab.com/atomic_clock/atomic_clock.html
416
http://www.info-pack.com/datetray/
417
http://www.lux-aeterna.com/clock/
418
http://www.enins.com/timesled/
419
http://www.acutesoftware.com.au/timer.html
420
http://www.timeutilites.com/
421
http://www.nehuenmultimedia.com.ar/html/atomic_clock.html
422
http://www.softdemon.com/clock/
423
http://www.timezonesforpcs.com/
424
http://www.xnotestopwatch.com/
425
http://visualversion.com/timebuddy/index.html
426
http://www.rpoint.com/pp/alarm-pro.aspx
427
http://www.lmhsoft.com/timesync/
428
http://www.vioio.com/mp3alarmclock.htm
429
http://littlemulticlock.com/littlemulticlock/files/littlemulticlock.html
430
http://www.shelltoys.com/s

615
http://www.uvm.edu/~jdion/
616
http://www.uvm.edu/~dmatthew/
617
http://www.uvm.edu/academics/catalogue2006-07/
618
http://optimization.asu.edu/~xue/
619
http://universitycommunications.uvm.edu/spring2001/webreleaseonunionpetitionresponse.htm
620
http://www.unitedacademics.org/
621
http://voyager.uvm.edu/
622
http://www.uvm.edu/whale/
623
http://www.uvm.edu/~fleming/
624
http://library.uvm.edu/
625
http://library.uvm.edu/dana/
626
http://www.uvm.edu/perkins/
627
http://www.uvm.edu/~pmrc/pmrccam.html
628
http://universitycommunications.uvm.edu/tour/?page=buildings
629
http://www.uvm.edu/~pmrc/sugarcam.html
630
http://www.uvm.edu/map/
631
http://www.uvm.edu/about_uvm/visiting/?page=greencam.html
632
http://www.uvm.edu/uvmtoday/
633
http://universitycommunications.uvm.edu/vq/index.html
634
http://universitycommunications.uvm.edu/
635
http://www.vermontcynic.com/
636
http://www.uvm.edu/~vtconn/
637
http://www.uvm.edu/~uvmemu/
638
http://www.uvm.edu/~uvmtv/
639
http://www.uvm.edu/~wruv/

816
http://migratorybirds.fws.gov/issues/cormorant/greatlakes_q&amp;a.htm
817
http://web.tiscali.it/sv2001/
818
http://nzbirds.com/birds/kawau.html
819
http://www.coldwaterimages.com/brandt_s_cormorant.html
820
http://www.kenyabirds.org.uk/great_cormorant.htm
821
http://www.kenyabirds.org.uk/cormorant-lt.htm
822
http://animaldiversity.ummz.umich.edu/site/accounts/information/sula_nebouxii.html
823
http://bioweb.uwlax.edu/bio203/s2008/kops_aman/
824
http://www.fws.gov/midway/mabo.html
825
http://en.wikipedia.org/wiki/sula_(bird)
826
http://www.fws.gov/midway/brbo.html
827
http://www.fws.gov/midway/rfbo.html
828
http://www.nzbirds.com/birds/takapu.html
829
http://en.wikipedia.org/wiki/darter
830
http://www.kenyabirds.org.uk/darter.htm
831
http://ibc.hbw.com/ibc/phtml/familia.phtml?idfamilia=18
832
http://www.thebigzoo.com/animals/lesser_flamingo.asp
833
http://www.geocities.com/flamingo22701/flamingo_mania
834
http://www.geocities.com/petsburgh/1975/flamingo.html
835
http://animaldiversi

964
http://www.kakapo.net/
965
http://en.wikipedia.org/wiki/kakapo
966
http://nzbirds.com/birds/kakapo.html
967
http://www.kakaporecovery.org.nz/
968
http://bioweb.uwlax.edu/bio203/s2008/runde_sama/
969
http://www.kenyabirds.org.uk/parrot-ob.htm
970
http://www.wingscentral.org/aps/faqcapes.aspx
971
http://www.wingscc.com/aps/poicep.htm
972
http://www.seabirds.org/penguins.htm
973
http://www.penguin.net.nz/
974
http://www.bluemoon.net/~opus/nframe/penguin.html
975
http://octopus.gma.org/surfing/antarctica/penguin.html
976
http://www.antarcticconnection.com/antarctic/wildlife/penguins/index.shtml
977
http://www.penguins.cl/
978
http://www.falklands.net/
979
http://englishnf.pinguins.info/
980
http://www.gdargaud.net/antarctica/penguins.html
981
http://www.adelie.pwp.blueyonder.co.uk/


In [13]:
label = []
for i in range(0,rows):
    label.append(0)
    

In [14]:
d={'Protocol':pd.Series(protocol),'Domain':pd.Series(domain),'Path':pd.Series(path),'Having_IP':pd.Series(having_ip),
   'URL_Length':pd.Series(len_url),'Having_@_symbol':pd.Series(having_at_symbol),
   'Redirection_//_symbol':pd.Series(redirection_symbol),'Prefix_suffix_separation':pd.Series(prefix_suffix_separation),
   'Sub_domains':pd.Series(sub_domains),'tiny_url':pd.Series(tiny_url),'web_traffic' : pd.Series(web_traffic) ,
   'domain_registration_length':pd.Series(domain_registration_length),'dns_record':pd.Series(dns_record),
   'statistical_report':pd.Series(statistical_report),'age_domain':pd.Series(age_domain),'http_tokens':pd.Series(http_tokens),
   'label':pd.Series(label)}
data=pd.DataFrame(d)
data

,Protocol,Domain,Path,Having_IP,URL_Length,Having_@_symbol,Redirection_//_symbol,Prefix_suffix_separation,Sub_domains,tiny_url,web_traffic,domain_registration_length,dns_record,statistical_report,age_domain,http_tokens,label
0,http,www.liquidgeneration.com,/,0,0,0,0,0,0,0,1,1,1,1,1,0,0
1,http,www.ceres.dti.ne.jp,/~nekoi/senno/senfirst.html,0,0,0,0,0,1,0,0,1,1,0,1,0,0
2,http,www.galeon.com,/kmh/,0,0,0,0,0,0,0,0,1,1,0,1,0,0
3,http,www.fanworkrecs.com,/,0,0,0,0,0,0,0,1,1,1,1,1,0,0
4,http,www2.117.ne.jp,/~mb1996ax/enadc.html,0,0,0,0,0,1,0,1,1,1,0,1,0,0
5,http,archive.rhps.org,/fritters/yui/index.html,0,0,0,0,0,2,0,1,1,1,0,1,0,0
6,http,www.cutepet.org,/,0,0,0,0,0,0,0,1,1,1,1,1,0,0
7,http,www.taremeparadise.com,/,0,0,0,0,0,0,0,1,1,1,1,1,0,0
8,http,www.internetdump.com,/users/pornographite/index1.html,0,2,0,0,0,2,0,1,1,1,1,1,0,0
9,http,darkkaminari.net,,0,0,0,0,0,0,0,1,1,1,1,1,0,0


In [16]:
data.to_csv("J_extracted_legitimate-urls.csv",index=False,encoding='UTF-8')